In [85]:
## five data cleaning techniques
# 1.Identify missing values, outliers, and inconsistencies.
# 2.Clean missing data, remove duplicates, standardize text.
# 3.Handle outliers and convert data types.
# 4.Create new features, normalize data, and encode categorical variables.
# 5.Save the cleaned and transformed dataset.

In [87]:
#Functions
import re
import pandas as pd
import numpy as np

#remove the word "Reported" and keep the date
def clean_dates(input_string):
    input_data = str(input_string)
    cleaned_string = re.sub(r'Reported\s*', '', input_data)
    return cleaned_string 
    
#Replace two numbers which has & with average result of two numbers
def clean_year_column(input_string):
    input_data = str(input_string)
    return re.sub(r'(\d{4})\.0', r'\1', input_data)

def replace_with_average(age_str):
    # Regex to find "x & y"
    age_str = str(age_str) 
    match = re.match(r'(\d+)\s*(?:and|or|to|&|/)\s*(\d+)', age_str)
    
    if match:
        num1 = int(match.group(1))
        num2 = int(match.group(2))
        average = (num1 + num2) / 2
        return average
    return age_str

#Find all numbers (including decimals) in the string
def extract_numbers(value):
    value = str(value).strip()
    numbers = re.findall(r'\d+(?:\.\d+)?', value)
    
    if numbers:
        return int(round(float(numbers[0])))
    return np.nan
    
#remove ½ and round up the numbers
def clean_age(age_str):
    age_str = str(age_str).strip()
    match = re.findall(r'\d+\.?\d*|\d+½', age_str)
    
    if match:
        processed_numbers = []
        for num in match:
            if '½' in num:
                num = num.replace('½', '.5') #Eg. 6½ is round up as 6.5
            processed_numbers.append(float(num))
        processed_numbers = [num for num in processed_numbers]
        
        if processed_numbers:
            return sum(processed_numbers) / len(processed_numbers)
    
    return None

#Keep sex/gender as F and M and replace other with empty value
def clean_gender_column(input_string):
    input_str = str(input_string)
    return re.sub(r'[^FM]', '', input_str)

#Keep name replace female|nan|NaN with empty value
def remove_female_and_nan(input_string):
    input_string = str(input_string)
    cleaned_str = re.sub(r'\b(male|female|nan|NaN)\b', '', input_string, flags=re.IGNORECASE)
    cleaned_str = re.sub(r'\s*,\s*', ',', cleaned_str)  
    cleaned_str = re.sub(r',+', ',', cleaned_str)       
    cleaned_str = cleaned_str.strip(', ')
    return cleaned_str
    
#########################################  Load and inspect raw data file ###########################################

##Read excel file
url = 'https://www.sharkattackfile.net/spreadsheets/GSAF5.xls'
df = pd.read_excel(url)


df['Date'] = df['Date'].apply(clean_dates)

df['Year'] = df['Year'].apply(clean_year_column)

############################################  Type Column  #########################################################
df["Type"] = df["Type"].str.strip()
df["Type"] = df["Type"].replace(['Unverified', 'Questionable', 'Under investigation'], 'Unconfirmed')
df["Type"] = df["Type"].replace('?', np.nan)


############################################  Activity Column  ####################################################

df['Activity'] = df['Activity'].astype(str).replace('nan', '')
df['Activity'] = df['Activity'].str.replace(r'^\d+$', '', regex=True)


############################################  Name Column  ####################################################
df['Name'] = df['Name'].apply(remove_female_and_nan)

############################################  Sex Column  #########################################################
df['Sex'] = df['Sex'].apply(clean_gender_column)

############################################  Age Column  #########################################################

#Replace two numbers which has & with average result of two numbers
df['Age'] = df['Age'].apply(replace_with_average)

#Find all numbers (including decimals) in the string
df['Age'] = df['Age'].apply(extract_numbers).astype(float).round().astype('Int64')

#remove ½ and keep only numbers
df['Age'] = df['Age'].apply(clean_age).astype(float).round().astype('Int64')

#Replace <NA> with empty values
df['Age'] = df['Age'].astype(str).replace('<NA>', '')

############################################  Injury Column  #########################################################
df["Injury"] = df["Injury"].str.strip()
# rename to fatal if fatal is in string
df["Injury"] = df["Injury"].apply(lambda x: "Fatal" if pd.notna(x) and "FATAL" in x.upper() and "NOT FATAL" not in x.upper() else x)

# rename non Fatal if "Fatal" not in string
df["Injury"] = df["Injury"].apply(lambda x: "Non-Fatal" if x != "Fatal" else x) 

############################################  Species Column  #########################################################
df = df.rename(columns={'Species ': 'Species'})
df = df.rename(columns={'original order': 'Original Order'})

#Save cleaned data file
df.to_csv('/Users/rishikeshdhokare/Documents/Ironhack/MiniProject/Quest2-Shark-Attacks/data-cleaning-pandas/sharkattacks_cleaned_data.csv', index=False)
df

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,...,Species,Source,pdf,href formula,href,Case Number,Case Number.1,Original Order,Unnamed: 21,Unnamed: 22
0,15 Mar 2024,2024,Unprovoked,AUSTRALIA,Queensland,Bargara Beach,Swimming,Brooklyn Sauer,F,13,...,Tiger shark,"Yahoo News, 3/15/2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,04 Mar 2024,2024,Unprovoked,USA,Hawaii,"Old Man's, Waikiki",Surfing,Matthew White,M,,...,Tiger shark 8',"Surfer, 3/6/2024F",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,02 Mar-2024,2024,Unprovoked,USA,Hawaii,"Rainbows, Oahu",Swimming,,F,11,...,3' to 4' shark,"Hawaii News Now, 3/4/2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,25 Feb-2024,2024,Unprovoked,AUSTRALIA,Western Australia,"Sandlnd Island, Jurian Bay",,,F,46,...,Tiger shark,"WA Today, 2/26/2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14 Feb-2024,2024,Unprovoked,INDIA,Maharashtra,"Vaitarna River, Palghar District",Fishing,Vicky Suresh Govari,M,32,...,"Bull shark, 7'","Times of India, 2/14/2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6964,nan,nan,NaN,NaN,NaN,NaN,,,,,...,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,NaN,NaN,NaN,NaN,NaN,NaN
6965,nan,nan,NaN,NaN,NaN,NaN,,,,,...,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,NaN,NaN,NaN,NaN,NaN,NaN
6966,nan,nan,NaN,NaN,NaN,NaN,,,,,...,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,NaN,NaN,NaN,NaN,NaN,NaN
6967,nan,nan,NaN,NaN,NaN,NaN,,,,,...,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df["Unnamed: 11"].unique()

In [ ]:
df["Unnamed: 21"].unique()

In [ ]:
df["Unnamed: 22"].unique()

In [89]:
import re

time_list = list(df["Time"].unique())
pattern = r'\d{2}h\d{2}'

time_list = [item for item in time_list if isinstance(item, str) and re.search(pattern, item)]
# time_list = [item.split() for item in time_list]

# time_list = df['Time'].str.contains(r'\d{2}h\d{2}', regex=True)
time_list


replaced_list = []

for time in time_list:
    if "-" in time:
        replaced_list.append(time)

replaced_list

df["Time"] = df["Time"].replace({'-16h30':'16h30',
 '09h00-10h00':'09h30',
 '14h00-15h00':'14h30',
 '14h00  -15h00':'14h30',
 '10h45-11h15':'11h00',
 '07h00 - 08h00':'07h30',
 '18h15-18h30':'18h30',
 '09h00 - 09h30':'09:15',
 '10h00 -- 11h00':'10h30',
 '09h00 -10h00':'09h30',
 '14h00 - 15h00':'14h40',
 '03h45 - 04h00':'04h00',
 '11h01 -time of ship sinking':'11h00',
 'Ship aban-doned at 03h10':'03h00',
 '06h00 -- 07h00':'06h30',
 '17h00-18h00':'17h30',
 '19h00-20h00':'19h30'})

df["Time"].unique()

replaced_list_1 = []


for time in df["Time"]:
    # Check if the 'time' is not NaN and does not match the pattern using re.search()
    if pd.notna(time) and not re.search(pattern, str(time)):  # Check for NaN and regex match
        replaced_list_1.append(time)

len(replaced_list_1)
set(replaced_list_1)

df["Time"] = df["Time"].replace({
 '"After dark"':'21h30',
 '"After lunch"':'13h00',
 '"Early evening"':'18h00',
 '"Evening"':'20h00',
 '"Midday"':'12h00',
 '"Night"':'22h00',
 '"shortly before dusk"':'06h00',
 'After Dusk':'09h00',
 'After dusk':'09h00',
 'After midnight':'01h00',
 'After noon':'13h00',
 'Afternoon':'15h00',
 'Before daybreak':'06h00',
 'Dark':'22h00',
 'Dawn':'20h00',
 'Daybreak':'08h00',
 'Daytime':'12h00',
 'Dusk':'08:00',
 'Early  morning':'07h00',
 'Early Morning':'07h00',
 'Early afternoon':'14h00',
 'Early morning':'07h00',
 'Evening':'20h00',
 'Just before dawn':'20h00',
 'Just before noon':'11h00',
 'Just before sundown':'19h00',
 'Late Afternoon':'17h00',
 'Late afternon':'17h00',
 'Late afternoon':'17h00',
 'Late morning':'10h00',
 'Late night':'23h00',
 'Lunchtime':'12h00',
 'Mid afternoon':'15h00',
 'Mid morning':'15h00',
 'Mid-morning':'15h00',
 'Midday':'12h00',
 'Midday.':'12h00',
 'Midnight':'24h00',
 'Morning':'08:00',
 'Morning ':'08:00',
 'Night':'22:00',
 'Nightfall':'21:00',
 'Noon':'12h00',
 'Shortly after midnight':'01:00',
 'Sunset':'20h00',
 'dusk':'08h00',
 'night':'22h00','01:00':'01h00',
 '0500':'05h00',
 '06j00':'22h00',
 '0830':'08h30',
 '08:00':'08h00',
 '09:15':'09h15',
 '10j30':'10h30',
 '10jh45':'10h45',
 '11hoo':'11h00',
 1300:'13h00',
 1415:'14h15',
 1500:'15h00',
 '15j45':'15h45',
 '1600':'16h00',
 '20:00':'20h00',
 '21:00':'21h00',
 '22:00':'22h00',
 '8:04 pm':'20h00'})

df["Time"].unique()

for time in df["Time"]:
    # Check if the 'time' is not NaN and does not match the pattern using re.search()
    if pd.notna(time) and not re.search(pattern, str(time)):  # Check for NaN and regex match
        replaced_list_1.append(time)

set(replaced_list_1)

{' ',
 '  ',
 '   ',
 '"After dark"',
 '"After lunch"',
 '"Early evening"',
 '"Evening"',
 '"Midday"',
 '"Night"',
 '"shortly before dusk"',
 '--',
 '01:00',
 '0500',
 '06j00',
 '0830',
 '08:00',
 '09:15',
 '10j30',
 '10jh45',
 '11hoo',
 1300,
 1415,
 1500,
 '15j45',
 '1600',
 '2 hours after Opperman',
 '2 hrs before sunset',
 '21:00',
 '22:00',
 '30 minutes after 1992.07.08.a',
 '8:04 pm',
 '9h00',
 'A.M.',
 'AM',
 'After Dusk',
 'After dusk',
 'After midnight',
 'After noon',
 'Afternoon',
 'Before daybreak',
 'Dark',
 'Dawn',
 'Daybreak',
 'Daytime',
 'Dusk',
 'Early  morning',
 'Early Morning',
 'Early afternoon',
 'Early morning',
 'Evening',
 'FATAL  (Wire netting installed at local beaches after this incident.)',
 'Just before dawn',
 'Just before noon',
 'Just before sundown',
 'Late Afternoon',
 'Late afternon',
 'Late afternoon',
 'Late morning',
 'Late night',
 'Lunchtime',
 'Mid afternoon',
 'Mid morning',
 'Mid-morning',
 'Midday',
 'Midday.',
 'Midnight',
 'Morning',
 'Mo

In [ ]:
pattern = r'\d{2}h\d{2}'

matched_items = [item for item in time_list if re.search(pattern, item)]

In [ ]:
# Findings

1. Species column has most of the values as null
2. Time stamp in Time coloumn is different
3. Name column has values in of sex(male, female) other than the actual name
4. Injury columns has many FATAL name injuries
5. ALl rows in cloumns have some null values
6. Age column should be numeric 
7. Age column has no numbers it is fllowed by string values 
8.  All the below columns are null/empty
href               object
Case Number        object
Case Number.1      object
original order    float64
Unnamed: 21        object
Unnamed: 22        object‚

9. There are no lables for 3 columns-- Unnamed: 11 , Unnamed: 21, Unnamed: 22
10. Many duplicate values in some of the columns

---
"Dates" 
Different formats, '15 Mar 2024' / '1883-1889' / nan / "0"
---
"Year"
Just three weird values
---
"Type"
Unprovoked', ' Provoked', 'Provoked', 'Questionable', 'Watercraft', 'Sea Disaster', nan, '?', 'Unconfirmed', 'Unverified', 'Invalid', 'Under investigation', 'Boat'
---
"Country"
Country name USA not unified, some NaN values
---
"Location"
Some numbers and NaN values instead of city/location names
---
"Activity"

---
"Name"
A wild mix of different attributes
---
"Sex"
Mostly M anf F, rest can probably be deleted
---
"Age"
Some ranges and words, mostly numbers of ages though
---
"Activity"